In [1]:
from AttentionModule import Conv2d_Attn

import torch
from torch import nn
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import re
import numpy as np

In [2]:
resnet_pretrained = models.resnet50(pretrained=True)

In [3]:
nn.Conv2d = Conv2d_Attn
resnet_attn = models.resnet50()

In [4]:
resnet_attn.load_state_dict(resnet_pretrained.state_dict(), strict=False)

In [5]:
# Change batchnorm behavior
# resnet_attn = resnet_attn.eval() 
# Don't want to do that because bn needs to be re-trained as well

In [6]:
# This block turns 'layer1.0.downsample.0.weight' to 'layer1[0].downsample[0].weight'
param_keys = list(resnet_attn.state_dict().keys())
formatted_keys = []
for k in param_keys:
    found = re.findall(r'\.[\d]{1,2}\.', k)
    if len(found):
        for f in found:
            k = k.replace(f, '[{}].'.format(f.strip('.')))
    formatted_keys.append(k)

In [7]:
# This block turn off gradient up for all params except attn_weights
def turn_off_grad_except(lst=[]):
    for k in formatted_keys:
        obj = eval('resnet_attn.'+k)
        for kw in lst:
            if not kw in k:
                obj.requires_grad = False
            else:
                obj.requires_grad = True

In [8]:
resnet_attn.fc = nn.Linear(resnet_attn.fc.in_features, 144)

Start training

In [9]:
batch_size = 32

In [10]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [11]:
total_imgs = len(trainset.imgs)

In [12]:
resnet_attn = resnet_attn.cuda()

In [13]:
total_attn_params = 0
for k in formatted_keys:
    obj = eval('resnet_attn.'+k)
    if 'attn_weights' in k:
        total_attn_params += np.prod(obj.shape)
print("Total number of attention parameters", total_attn_params)

Total number of attention parameters 26560


We want the attention parameters to diverge from 1, therefore we penalize element-wise square loss as $\lambda (1 \times \text{# params} - (x - 1)^2)$

In [14]:
_lambda = 1e-2

In [15]:
def get_params_objs(name, net='resnet_attn'):
    res = []
    for k in formatted_keys:
        obj = eval(f'{net}.'+k)
        if name in k:
            res.append(obj)
    return res

In [16]:
def compute_attn_loss(n_params=26560):
    attns = get_params_objs('attn_weights')
    penality = sum([torch.pow(t - 1,2).mean() for t in attns])
    return _lambda*(n_params - penality)

In [17]:
print_every = 5

In [18]:
def train_one_epoch(add_attn=True):
    cls_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet_attn.parameters()))
    
    running_loss = 0.0
    running_attn_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

        optimizer.zero_grad()
        outputs = resnet_attn(inputs)
        loss = cls_criterion(outputs, labels)
        attn_loss = compute_attn_loss()
        if add_attn:
            loss += attn_loss

        loss.backward()
        optimizer.step()


        running_loss += loss.data[0]
        running_attn_loss += attn_loss.data[0]

        if i % print_every == 0:
            print('[%5d] iter, [%2f] epoch, avg loss: %.3f, attn_loss: %.5f ' %
                  (i + 1, i*batch_size/total_imgs, running_loss/print_every, running_attn_loss/print_every))
            running_loss = 0.0
            running_attn_loss = 0.0

Train a fresh fc layer. 
`turn_off_grad_except([])` turns off grads for all weights but the fc layer

In [19]:
turn_off_grad_except(['fc'])
resnet_attn.eval() # Turn off batchnorm
train_one_epoch(add_attn=False)

[    1] iter, [0.000000] epoch, avg loss: 1.032, attn_loss: 53.12000 
[    6] iter, [0.019605] epoch, avg loss: 4.804, attn_loss: 265.60001 
[   11] iter, [0.039211] epoch, avg loss: 4.892, attn_loss: 265.60001 
[   16] iter, [0.058816] epoch, avg loss: 4.647, attn_loss: 265.60001 
[   21] iter, [0.078422] epoch, avg loss: 4.314, attn_loss: 265.60001 
[   26] iter, [0.098027] epoch, avg loss: 4.091, attn_loss: 265.60001 
[   31] iter, [0.117633] epoch, avg loss: 3.889, attn_loss: 265.60001 
[   36] iter, [0.137238] epoch, avg loss: 3.989, attn_loss: 265.60001 
[   41] iter, [0.156844] epoch, avg loss: 3.793, attn_loss: 265.60001 
[   46] iter, [0.176449] epoch, avg loss: 3.981, attn_loss: 265.60001 
[   51] iter, [0.196054] epoch, avg loss: 3.458, attn_loss: 265.60001 
[   56] iter, [0.215660] epoch, avg loss: 3.794, attn_loss: 265.60001 
[   61] iter, [0.235265] epoch, avg loss: 3.646, attn_loss: 265.60001 
[   66] iter, [0.254871] epoch, avg loss: 3.412, attn_loss: 265.60001 
[   71]

In [20]:
from tqdm import tqdm

In [21]:
def score(net=resnet_attn, batch_size=batch_size):
    trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    valset = torchvision.datasets.ImageFolder(root='./data/val', transform=transform)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    train_correct = 0
    val_correct = 0
    
    for inp, label in tqdm(iter(trainloader)):
        _, idx = net(Variable(inp).cuda()).max(1)
        train_correct += int(sum(idx.cpu().data == label))
    
    for inp, label in tqdm(iter(valloader)):
        _, idx = net(Variable(inp).cuda()).max(1)
        val_correct += int(sum(idx.cpu().data == label))
    
    return {
        'train_accu': train_correct/len(trainset),
        'val_accu': val_correct/len(valset)
    }

In [22]:
score(batch_size=64)

100%|██████████| 28/28 [00:13<00:00,  2.06it/s]


{'train_accu': 0.38377649797818897, 'val_accu': 0.33352337514253133}

In [23]:
turn_off_grad_except(['attn_weights','bn'])
resnet_attn = resnet_attn.train()
_lambda = 1e-4
train_one_epoch(add_attn=True)

[    1] iter, [0.000000] epoch, avg loss: 1.207, attn_loss: 0.53120 
[    6] iter, [0.019605] epoch, avg loss: 5.745, attn_loss: 2.65600 
[   11] iter, [0.039211] epoch, avg loss: 5.688, attn_loss: 2.65600 
[   16] iter, [0.058816] epoch, avg loss: 5.379, attn_loss: 2.65600 
[   21] iter, [0.078422] epoch, avg loss: 5.470, attn_loss: 2.65600 
[   26] iter, [0.098027] epoch, avg loss: 5.416, attn_loss: 2.65600 
[   31] iter, [0.117633] epoch, avg loss: 5.480, attn_loss: 2.65600 
[   36] iter, [0.137238] epoch, avg loss: 5.273, attn_loss: 2.65600 
[   41] iter, [0.156844] epoch, avg loss: 5.344, attn_loss: 2.65600 
[   46] iter, [0.176449] epoch, avg loss: 5.305, attn_loss: 2.65600 
[   51] iter, [0.196054] epoch, avg loss: 5.274, attn_loss: 2.65600 
[   56] iter, [0.215660] epoch, avg loss: 5.231, attn_loss: 2.65600 
[   61] iter, [0.235265] epoch, avg loss: 5.209, attn_loss: 2.65600 
[   66] iter, [0.254871] epoch, avg loss: 5.318, attn_loss: 2.65600 
[   71] iter, [0.274476] epoch, av

In [24]:
score(batch_size=64)

100%|██████████| 28/28 [00:14<00:00,  1.97it/s]


{'train_accu': 0.4773924764122044, 'val_accu': 0.40250855188141393}

In [28]:
turn_off_grad_except(['attn_weights'])
resnet_attn = resnet_attn.eval()
_lambda = 1
train_one_epoch(add_attn=True)
score(batch_size=64)

[    1] iter, [0.000000] epoch, avg loss: 5312.389, attn_loss: 5312.00000 
[    6] iter, [0.019605] epoch, avg loss: 26562.083, attn_loss: 26560.00000 
[   11] iter, [0.039211] epoch, avg loss: 26562.094, attn_loss: 26560.00000 
[   16] iter, [0.058816] epoch, avg loss: 26562.080, attn_loss: 26559.99883 
[   21] iter, [0.078422] epoch, avg loss: 26562.244, attn_loss: 26559.99805 
[   26] iter, [0.098027] epoch, avg loss: 26562.171, attn_loss: 26559.99805 
[   31] iter, [0.117633] epoch, avg loss: 26562.136, attn_loss: 26559.99805 
[   36] iter, [0.137238] epoch, avg loss: 26561.817, attn_loss: 26559.99609 
[   41] iter, [0.156844] epoch, avg loss: 26561.794, attn_loss: 26559.99609 
[   46] iter, [0.176449] epoch, avg loss: 26562.011, attn_loss: 26559.99570 
[   51] iter, [0.196054] epoch, avg loss: 26561.921, attn_loss: 26559.99414 
[   56] iter, [0.215660] epoch, avg loss: 26562.111, attn_loss: 26559.99414 
[   61] iter, [0.235265] epoch, avg loss: 26562.123, attn_loss: 26559.99219 
[

100%|██████████| 28/28 [00:13<00:00,  2.04it/s]


{'train_accu': 0.5192991055017767, 'val_accu': 0.427594070695553}

In [31]:
resnet_attn.conv1.attn_weights.cpu().data.numpy().squeeze()

array([0.9920831 , 0.9934335 , 1.0174953 , 1.0028536 , 0.9993886 ,
       0.99619156, 1.0068909 , 1.0230352 , 0.9964805 , 0.99543005,
       1.0036072 , 0.97201735, 1.0118227 , 1.        , 1.0046093 ,
       1.0039697 , 0.9995543 , 1.0046791 , 1.0027827 , 1.0176603 ,
       1.02211   , 1.001126  , 1.0142523 , 1.0215573 , 1.0014066 ,
       0.9899166 , 1.0183663 , 1.0128137 , 1.0150323 , 1.0450556 ,
       1.0052605 , 0.82890725, 1.0075049 , 1.0083247 , 1.0341356 ,
       0.9981748 , 1.0052088 , 0.98948526, 1.0258948 , 0.99799013,
       0.99088335, 0.9929585 , 0.97616553, 1.0020766 , 1.009043  ,
       0.99546564, 1.0090386 , 1.0029176 , 0.9877005 , 1.0169706 ,
       1.0088778 , 1.0115087 , 0.9861459 , 1.0229034 , 0.98577136,
       0.994887  , 0.99842876, 1.0175492 , 0.99294907, 0.9721664 ,
       0.9978678 , 1.0024883 , 1.0015049 , 1.0037984 ], dtype=float32)

Thoughts:
- It works. 
- Interleaving training works for now, but maybe it's fine if we just train it all together
- We need to find a better loss to let attention diverge faster